# Haystack and MongoDB Atlas RAG notebook

Install dependencies:

In [32]:
pip install haystack-ai mongodb-atlas-haystack tiktoken


## Setup MongoDB Atlas connection and Open AI


* Set the MongoDB connection string. Follow the steps [here](https://www.mongodb.com/docs/manual/reference/connection-string/) to get the connection string from the Atlas UI.

* Set the OpenAI API key. Steps to obtain an API key as [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)

In [6]:
import getpass
import os


In [7]:
os.environ['MONGO_CONNECTION_STRING']=getpass.getpass("Enter your MongoDB connection string:")

Enter your MongoDB connection string:··········


In [8]:
os.environ['OPENAI_API_KEY']=getpass.getpass("Enter your Open AI Key:")

Enter your Open AI Key:··········


## Create vector search index on collection

Follow this [tutorial](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-index/) to create a vector index on database: `haystack_test` collection `test_collection`.

Verify that the index name is `vector_index` and the syntax specify:
```
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
}
```

### Setup vector store to load documents:

In [29]:
from haystack import Pipeline, Document
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.writers import DocumentWriter
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever

# Create some example documents
documents = [
    Document(content="My name is Jean and I live in Paris."),
    Document(content="My name is Mark and I live in Berlin."),
    Document(content="My name is Giorgio and I live in Rome."),
]

document_store = MongoDBAtlasDocumentStore(
    database_name="haystack_test",
    collection_name="test_collection",
    vector_search_index="vector_index",
)

Build the writer pipeline to load documnets

In [12]:
# Setting up a document writer to handle the insertion of documents into the MongoDB collection.
doc_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

# Initializing a document embedder to convert text content into vectorized form.
doc_embedder = OpenAIDocumentEmbedder()

# Creating a pipeline for indexing documents. The pipeline includes embedding and writing documents.
indexing_pipe = Pipeline()
indexing_pipe.add_component(instance=doc_embedder, name="doc_embedder")
indexing_pipe.add_component(instance=doc_writer, name="doc_writer")

# Connecting the components of the pipeline for document flow.
indexing_pipe.connect("doc_embedder.documents", "doc_writer.documents")

# Running the pipeline with the list of documents to index them in MongoDB.
indexing_pipe.run({"doc_embedder": {"documents": documents}})


Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]


{'doc_embedder': {'meta': {'model': 'text-embedding-ada-002',
   'usage': {'prompt_tokens': 32, 'total_tokens': 32}}},
 'doc_writer': {'documents_written': 3}}

## Build a RAG Pipeline

Lets create a pipeline that will Retrieve Augment and Generate a response for user questions

In [33]:
# Template for generating prompts for a movie recommendation engine.
prompt_template = """
    You are an assistant allowed to use the following context documents.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \Query: {{query}}
    \nAnswer:
"""

# Setting up a retrieval-augmented generation (RAG) pipeline for generating responses.
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", OpenAITextEmbedder())

# Adding a component for retrieving related documents from MongoDB based on the query embedding.
rag_pipeline.add_component(instance=MongoDBAtlasEmbeddingRetriever(document_store=document_store,top_k=15, filters={}), name="retriever")

# Building prompts based on retrieved documents to be used for generating responses.
rag_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")

# Adding a language model generator to produce the final text output.
rag_pipeline.add_component(instance=OpenAIGenerator(), name="llm")

# Connecting the components of the RAG pipeline to ensure proper data flow.
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: MongoDBAtlasEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

Lets test the pipeline

In [34]:
query = "Where does mark live?"
result = rag_pipeline.run(
                                {
                                    "text_embedder": {"text": query},
                                    "prompt_builder": {"query": query},
                                });
print(result)

DocumentStoreError: Retrieval of documents from MongoDB Atlas failed: invalid parameter: expected an object (filter), full error: {'ok': 0.0, 'errmsg': 'invalid parameter: expected an object (filter)', 'code': 10065, 'codeName': 'Location10065', '$clusterTime': {'clusterTime': Timestamp(1716804481, 2), 'signature': {'hash': b'\xd0\xf5\xce`6]j\xb5\xbf\xf2CN\x03\x9a3\xb2HiC\x9c', 'keyId': 7317623845651742726}}, 'operationTime': Timestamp(1716804481, 2)}